In [ ]:
import datasets
import csv
import json
import numpy as np
import random
import itertools

PREDICTIONS_PATH = "../predictions"
MODEL_PAIRS = [
    ('bart_base_asqa_dpr@3', 'bart_large_asqa_dpr@3'),
    ('bart_large_asqa_dpr@3', 'bart_large_eli5_asqa_dpr@3'),
    ('t5_base_asqa_dpr@3', 't5_base_msmarco_asqa_dpr@3'),
    ('t5_base_asqa_dpr@3', 'bart_base_asqa_dpr@3'),
    ('bart_large_asqa_bm25@3', 'bart_large_asqa_dpr@3'),
    ('bart_large_asqa_bm25@1', 'bart_large_asqa_bm25@3'),
    ('bart_large_asqa_bm25@3', 'bart_large_asqa_bm25@5'),
    ('bart_large_asqa_dpr@1', 'bart_large_asqa_dpr@3'),
    ('bart_large_asqa_dpr@3', 'bart_large_asqa_dpr@5'),
    ('bart_large_eli5_asqa_dpr@3', 'reference'),
    ('dpr@1', 'bart_base_asqa_dpr@1'),
    ('dpr@1', 'bart_large_asqa_dpr@1'),
]
NUM_QUESTIONS = 4

# Load the dataset
with open('../dataset/ASQA.json', 'r') as f:
    asqa_dataset = json.load(f)
    asqa_dataset = asqa_dataset['dev']

In [ ]:
def get_question_predictions(model, key):
    if model == 'reference':
        # Get long answer from ASQA dataset
        return asqa_dataset[key]['annotations'][0]['long_answer']
    
    with open(f"{PREDICTIONS_PATH}/{model}.json") as f:
        predictions = json.load(f)
        return predictions[key]

def get_random_question(model1, model2):
    # Find question which predictions have at least one gold answer in them
    random_question = np.random.choice(list(asqa_dataset.keys()), 1)[0]
    
    gold_answers = set(itertools.chain(*[p['short_answers'] for p in asqa_dataset[random_question]['qa_pairs']]))
    prediction1 = get_question_predictions(model1, random_question)
    prediction2 = get_question_predictions(model2, random_question)
    gold_in_prediction1 = itertools.chain(*[g.split(" ") for g in gold_answers if g.lower() in prediction1.lower()])
    gold_in_prediction2 = itertools.chain(*[g.split(" ") for g in gold_answers if g.lower() in prediction2.lower()])
    
    if len(list(gold_in_prediction1)) > 0 or len(list(gold_in_prediction2)) > 0:
        asqa_dataset.pop(random_question)
        return random_question, prediction1, prediction2
    else:
        return get_random_question(model1, model2)

for (model1, model2) in MODEL_PAIRS:
    with open(f"./comparisons/{model1}_vs_{model2}_setup.tsv", "wt", encoding='cp1252') as out_file:
        # write to tsv file
        tsv_writer = csv.writer(out_file, delimiter='\t')
        tsv_writer.writerow(['key', 'model1', 'model2', 'prediction1', 'prediction2'])
        # select random questions from dataset
        for i in range(NUM_QUESTIONS):
            # shuffle models so they are in random order
            model1_random = random.choice((model1, model2))
            model2_random = model1 if model1_random == model2 else model2
            sample_id, prediction1, prediction2 = get_random_question(model1_random, model2_random)
            tsv_writer.writerow([sample_id, model1_random, model2_random, prediction1, prediction2])

In [ ]:
import os

#Make files ready for drive
for file in os.listdir("./comparisons"):
    out_file = file.replace("_setup", "")
    os.system(f"python preparation.py --asqa ../dataset/ASQA.json --setup ./comparisons/{file} --dst ./comparisons_rfd/{out_file}")